# NSAI Level 1B: Multi-Detector Neuro-Symbolic Intent Classification

**Architecture Shift:**
- **Level 1A**: Single multi-class classifier
- **Level 1B**: One binary detector per intent

**Core Concept:**
- Each detector independently scores: "Does this match MY intent?"
- Scores ∈ [0,1], do NOT sum to 1
- Rules (not models) decide final outcome

**Dataset**: utterance, intent (4 classes: investigate, execution, summarization, out_of_scope)

**Output**: Deterministic, explainable decisions with detector scores + rule governance

**Method**: Multi-detector statistical learning + symbolic rule precedence (no LLMs, no timestamps)

## Part 1: Data Preparation

In [ ]:
# Imports
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
# Load & Validate Data
df = pd.read_csv('../data/intents_base.csv')

# Assert columns
assert set(df.columns) == {'utterance', 'intent'}, f"Expected columns {{utterance, intent}}, got {set(df.columns)}"

# Normalize intent labels
df['intent'] = df['intent'].str.lower().str.strip()

# Show class distribution
print("Class Distribution:")
print(df['intent'].value_counts())
print(f"\nTotal: {len(df)} records")

In [ ]:
# Train/Test Split
X = df['utterance']
y = df['intent']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"Train: {len(X_train)} | Test: {len(X_test)}")

## Part 2: Multi-Detector Training

In [ ]:
# Intent-Specific Detector Models
# Train ONE binary classifier per intent

intents = ['investigate', 'execution', 'summarization', 'out_of_scope']
detectors = {}

print("Training binary detectors...\n")

for intent in intents:
    # Create binary labels: 1 if utterance matches this intent, 0 otherwise
    y_binary_train = (y_train == intent).astype(int)
    y_binary_test = (y_test == intent).astype(int)
    
    # Create detector pipeline
    detector = Pipeline([
        ('tfidf', TfidfVectorizer(
            max_features=5000,
            ngram_range=(1, 2),
            stop_words='english'
        )),
        ('classifier', LogisticRegression(
            solver='lbfgs',
            random_state=42,
            max_iter=1000
        ))
    ])
    
    # Train detector
    detector.fit(X_train, y_binary_train)
    
    # Evaluate
    y_pred = detector.predict(X_test)
    report = classification_report(y_binary_test, y_pred, output_dict=True)
    
    # Store detector
    detectors[intent] = detector
    
    print(f"{intent:>20} detector: Accuracy={report['accuracy']:.4f}, F1={report['1']['f1-score']:.4f}")

print(f"\nAll {len(detectors)} detectors trained")

## Part 3: Multi-Detector Signal Extraction

In [ ]:
# Multi-Detector Signal Extraction
def extract_detector_signals(text):
    """Extract independent scores from all intent detectors
    
    Returns:
        Dictionary with detector_scores, top_detector, margin, tokens
    """
    detector_scores = {}
    
    # Get score from each detector independently
    for intent, detector in detectors.items():
        # Get probability of positive class (utterance matches this intent)
        proba = detector.predict_proba([text])[0]
        # Score is probability of class=1 (match)
        detector_scores[intent] = float(proba[1])
    
    # Sort by score
    sorted_detectors = sorted(detector_scores.items(), key=lambda x: x[1], reverse=True)
    top_detector = sorted_detectors[0][0]
    top_score = sorted_detectors[0][1]
    second_detector = sorted_detectors[1][0]
    second_score = sorted_detectors[1][1]
    score_margin = top_score - second_score
    
    # Get token count from any detector's TF-IDF (all use same settings)
    first_detector = detectors[intents[0]]
    tfidf_vec = first_detector.named_steps['tfidf'].transform([text])
    active_features = tfidf_vec.toarray()[0]
    meaningful_tokens = int(np.sum(active_features > 0))
    
    return {
        'detector_scores': detector_scores,
        'top_detector': top_detector,
        'top_score': top_score,
        'second_detector': second_detector,
        'second_score': second_score,
        'score_margin': score_margin,
        'meaningful_tokens': meaningful_tokens
    }

print("Multi-detector signal extraction defined")

## Part 4: Level 1B Symbolic Rules

In [ ]:
# Level 1B Rule Configuration (Explicit)
BASE_MIN_SCORE = 0.50
AMBIGUITY_MARGIN = 0.10
EXECUTION_MIN_SCORE = 0.85
MIN_TOKENS_OUT_OF_SCOPE = 3

# Rule priority (highest first)
RULE_PRIORITY = [
    "R_OUT_OF_SCOPE",
    "R_EXEC_SAFETY",
    "R_AMBIGUOUS",
    "R_DEFAULT"
]

print("Level 1B Configuration:")
print(f"  BASE_MIN_SCORE: {BASE_MIN_SCORE}")
print(f"  AMBIGUITY_MARGIN: {AMBIGUITY_MARGIN}")
print(f"  EXECUTION_MIN_SCORE: {EXECUTION_MIN_SCORE}")
print(f"  MIN_TOKENS_OUT_OF_SCOPE: {MIN_TOKENS_OUT_OF_SCOPE}")
print(f"\nRule Priority: {RULE_PRIORITY}")

In [ ]:
# Level 1B Symbolic Rules (Plain, Deterministic)
def apply_level1b_rules(signals):
    """Apply deterministic rules to detector signals
    
    Rules operate ONLY on numeric signals.
    Rules ALWAYS override raw scores.
    
    Returns:
        Dictionary with predicted_intent, decision_state, decision_reason, triggered_rules
    """
    triggered_rules = []
    predicted_intent = signals['top_detector']  # Default
    decision_state = 'accepted'
    decision_reason = 'R_DEFAULT'
    
    detector_scores = signals['detector_scores']
    
    # Count how many detectors score above threshold
    above_threshold = sum(1 for score in detector_scores.values() if score >= BASE_MIN_SCORE)
    
    # R_OUT_OF_SCOPE (Highest Priority)
    if signals['meaningful_tokens'] < MIN_TOKENS_OUT_OF_SCOPE:
        triggered_rules.append({
            'rule_id': 'R_OUT_OF_SCOPE',
            'priority': 100,
            'condition': f"meaningful_tokens < {MIN_TOKENS_OUT_OF_SCOPE}",
            'value': signals['meaningful_tokens']
        })
        predicted_intent = 'out_of_scope'
        decision_state = 'blocked'
        decision_reason = 'R_OUT_OF_SCOPE'
        return {
            'predicted_intent': predicted_intent,
            'decision_state': decision_state,
            'decision_reason': decision_reason,
            'triggered_rules': triggered_rules
        }
    
    # Check if all scores are below threshold
    if all(score < BASE_MIN_SCORE for score in detector_scores.values()):
        triggered_rules.append({
            'rule_id': 'R_OUT_OF_SCOPE',
            'priority': 100,
            'condition': f"all scores < {BASE_MIN_SCORE}",
            'value': max(detector_scores.values())
        })
        predicted_intent = 'out_of_scope'
        decision_state = 'blocked'
        decision_reason = 'R_OUT_OF_SCOPE'
        return {
            'predicted_intent': predicted_intent,
            'decision_state': decision_state,
            'decision_reason': decision_reason,
            'triggered_rules': triggered_rules
        }
    
    # R_EXEC_SAFETY
    exec_score = detector_scores['execution']
    if exec_score >= BASE_MIN_SCORE and exec_score < EXECUTION_MIN_SCORE:
        triggered_rules.append({
            'rule_id': 'R_EXEC_SAFETY',
            'priority': 90,
            'condition': f"execution_score >= {BASE_MIN_SCORE} AND < {EXECUTION_MIN_SCORE}",
            'value': exec_score
        })
        
        # Check if investigate is viable alternative
        if detector_scores['investigate'] >= BASE_MIN_SCORE:
            predicted_intent = 'investigate'
            decision_state = 'accepted'
            decision_reason = 'R_EXEC_SAFETY'
        else:
            predicted_intent = 'execution'
            decision_state = 'needs_clarification'
            decision_reason = 'R_EXEC_SAFETY'
        
        return {
            'predicted_intent': predicted_intent,
            'decision_state': decision_state,
            'decision_reason': decision_reason,
            'triggered_rules': triggered_rules
        }
    
    # R_AMBIGUOUS
    if above_threshold >= 2 and signals['score_margin'] < AMBIGUITY_MARGIN:
        triggered_rules.append({
            'rule_id': 'R_AMBIGUOUS',
            'priority': 50,
            'condition': f"multiple scores >= {BASE_MIN_SCORE} AND margin < {AMBIGUITY_MARGIN}",
            'value': signals['score_margin']
        })
        predicted_intent = signals['top_detector']
        decision_state = 'needs_clarification'
        decision_reason = 'R_AMBIGUOUS'
        return {
            'predicted_intent': predicted_intent,
            'decision_state': decision_state,
            'decision_reason': decision_reason,
            'triggered_rules': triggered_rules
        }
    
    # R_DEFAULT - Select highest scoring detector
    triggered_rules.append({
        'rule_id': 'R_DEFAULT',
        'priority': 10,
        'condition': 'default to highest scoring detector',
        'value': signals['top_score']
    })
    predicted_intent = signals['top_detector']
    decision_state = 'accepted'
    decision_reason = 'R_DEFAULT'
    
    return {
        'predicted_intent': predicted_intent,
        'decision_state': decision_state,
        'decision_reason': decision_reason,
        'triggered_rules': triggered_rules
    }

print("Level 1B symbolic rules defined")

## Part 5: Final Decision Function

In [ ]:
# Final Decision Function
def level1b_predict(text):
    """Complete Level 1B inference pipeline
    
    Returns:
        Structured decision with detector scores, rules, and final outcome
    """
    # Extract detector signals
    signals = extract_detector_signals(text)
    
    # Apply symbolic rules
    rules_output = apply_level1b_rules(signals)
    
    # Build final output
    return {
        'predicted_intent': rules_output['predicted_intent'],
        'decision_state': rules_output['decision_state'],
        'decision_reason': rules_output['decision_reason'],
        'detector_scores': signals['detector_scores'],
        'top_detector': signals['top_detector'],
        'score_margin': signals['score_margin'],
        'meaningful_tokens': signals['meaningful_tokens'],
        'triggered_rules': rules_output['triggered_rules']
    }

print("Level 1B prediction function ready")

## Part 6: Explainable Demo

In [ ]:
# Explainable Demo Output
test_utterances = [
    "why is server cpu high",
    "restart nginx on host123",
    "summarize the incident from yesterday",
    "hello",
    "delete all production databases",
    "server issues yesterday"
]

print("="*100)
print("LEVEL 1B: MULTI-DETECTOR NEURO-SYMBOLIC INFERENCE")
print("="*100)

for utterance in test_utterances:
    result = level1b_predict(utterance)
    
    print(f"\n{'-'*100}")
    print(f"UTTERANCE: {utterance}")
    print(f"{'-'*100}")
    
    # Detector scores
    print(f"\nDETECTOR SCORES:")
    for intent, score in sorted(result['detector_scores'].items(), key=lambda x: x[1], reverse=True):
        bar = '█' * int(score * 50)
        print(f"  {intent:>20}: {score:.4f} {bar}")
    
    print(f"\n  Top Detector: {result['top_detector']}")
    print(f"  Score Margin: {result['score_margin']:.4f}")
    print(f"  Meaningful Tokens: {result['meaningful_tokens']}")
    
    # Triggered rules
    print(f"\nTRIGGERED RULES:")
    for rule in sorted(result['triggered_rules'], key=lambda r: r['priority'], reverse=True):
        print(f"  [{rule['priority']:3d}] {rule['rule_id']:20s}: {rule['condition']}")
        print(f"       Signal Value: {rule['value']}")
    
    # Final decision
    print(f"\nFINAL DECISION:")
    print(f"  Predicted Intent: {result['predicted_intent']}")
    print(f"  Decision State: {result['decision_state']}")
    print(f"  Decision Reason: {result['decision_reason']}")
    
    # Structured JSON
    print(f"\nSTRUCTURED OUTPUT:")
    output_json = {
        'utterance': utterance,
        'predicted_intent': result['predicted_intent'],
        'decision_state': result['decision_state'],
        'decision_reason': result['decision_reason'],
        'detector_scores': result['detector_scores'],
        'score_margin': result['score_margin'],
        'meaningful_tokens': result['meaningful_tokens']
    }
    print(json.dumps(output_json, indent=2))

print(f"\n{'='*100}")
print("DEMO COMPLETE")
print("="*100)

## Validation: Test Set Evaluation

In [ ]:
# Evaluate Level 1B on test set
print("Evaluating Level 1B on test set...\n")

predictions = []
decision_states = []
rule_counts = {rule: 0 for rule in RULE_PRIORITY}

for text, true_intent in zip(X_test, y_test):
    result = level1b_predict(text)
    predictions.append(result['predicted_intent'])
    decision_states.append(result['decision_state'])
    
    # Count triggered rules
    for rule in result['triggered_rules']:
        rule_counts[rule['rule_id']] += 1

# Accuracy
accuracy = sum(p == t for p, t in zip(predictions, y_test)) / len(y_test)
print(f"Accuracy: {accuracy:.4f}")

# Decision state distribution
print(f"\nDecision State Distribution:")
from collections import Counter
state_counts = Counter(decision_states)
for state, count in state_counts.items():
    print(f"  {state:>20}: {count:3d} ({count/len(decision_states)*100:.1f}%)")

# Rule trigger counts
print(f"\nRule Trigger Counts:")
for rule in RULE_PRIORITY:
    count = rule_counts[rule]
    print(f"  {rule:>20}: {count:3d} ({count/len(X_test)*100:.1f}%)")

# Full classification report
print(f"\nClassification Report:")
print(classification_report(y_test, predictions))